In [1]:
import sys
from pathlib import Path
import numpy as np

current_dir = Path.cwd()
if str(current_dir) not in sys.path:
    sys.path.insert(0, str(current_dir))

print(current_dir)

from src import (
    DataConfig, DataLoader, ModelingStrategy, ReleaseManager, BenchmarkPipeline, create_config
)

np.random.seed(42)

/Users/ivn/Documents/PhD/Transformer Research/Code/Benchmarking


In [2]:
data_config = DataConfig(
    mapping_path = 'data/feature_mapping_train.pkl',
    features_path = "/Users/ivn/Documents/PhD/Transformer Research/Code/Benchmarking/data/db_snapshot_offsite/train_data/processed/train_data_features.feather",
    target_path = "/Users/ivn/Documents/PhD/Transformer Research/Code/Benchmarking/data/db_snapshot_offsite/train_data/train_data/train_data_target.feather",
    split_date="2016-01-01",
)

sku_tuples=[(1912, 7), (377, 1), (715, 7)]

quantiles = [0.5, 0.7, 0.9, 0.95, 0.99]
pipeline = BenchmarkPipeline(data_config)

In [3]:
loader = DataLoader(data_config)
data = loader.load_data()


In [ ]:

pipeline = BenchmarkPipeline(data_config)

# Step 1: Tune hyperparameters
tune_result = pipeline.run_experiment(
    sku_tuples=10,
    modeling_strategy=ModelingStrategy.COMBINED,
    model_type="xgboost_quantile",
    quantile_alphas=[0.7],
    mode="hp_tune",
    tune_on=50,
    tuning_config={'n_trials': 100, 'n_folds': 5}
)

In [4]:
results_lightning_std = pipeline.run_experiment(
    sku_tuples=sku_tuples,
    modeling_strategy=ModelingStrategy.INDIVIDUAL,
    model_type="xgboost_quantile",
    quantile_alphas=quantiles,
    hyperparameters = {
        "eta": 0.05,
        "max_depth": 8,
        "min_child_weight": 20,
        "subsample": 0.8,
        "colsample_bytree": 0.7,
        "gamma": 1.0,   
        "lambda": 10.0,
        "alpha": 1.0,
        "tree_method": "hist",
        "num_boost_round": 100,
        "seed": 42
},
    experiment_name="xgb_quantile_test",
    evaluate_on_test=True
)

Training models: 100%|██████████| 3/3 [00:01<00:00,  2.12it/s]


In [11]:
sample_result = results_lightning_std.training_results[10]
print(f"Model type: {sample_result.model_type}")
print(f"Strategy: {sample_result.modeling_strategy.value}")
print(f"SKU tuples: {sample_result.sku_tuples}")
print(f"Quantile level: {sample_result.quantile_level}")
print(f"quantile_score: {sample_result.performance_metrics.get('quantile_score', 'N/A')}")
print(f"predictions: {sample_result.performance_metrics.get('predictions', 'N/A')}")
print("overall stats: ----------------------------------------------------")
print(f"number of models trained: {len(results_lightning_std.training_results)}")

Model type: xgboost_quantile
Strategy: individual
SKU tuples: [(715, 7)]
Quantile level: 0.5
quantile_score: [0.  0.  0.  0.5 1.  0.5 0.  0.5 0.5 1.  0.  0.5 0.5 0.  1.  0.5 0.5 0.5
 0.  0.  1.  0.  0.  0.  1.  0.5 1.  0.5 0.  0.5 0.5 0.5 0.5 1.  0.5 1.5
 0.  0.5 0.  0.5 1.5 0.5 4.  0.5 0.  0.5 0.  1.  0.5 0.  0.5 0.5 1.5 0.
 0.5 0.5 0.5 0.5 2.  0.5 0.5 0.5 0.5 0.5 1.  0.  0.5 0.5 1.  0.5 1.5 0.5
 0.  0.5 0.5 0.5 0.  0.5 2.  0.  0.5 0.5 0.5 0.5 0.  0.  1.  0.  0.  0.5
 0.5 0.  0.5 0.  0.  0.5 1.  0.5 0.5 0.  1.  0.5 0.5 0.5 0.5 1.  1.  0.
 0.5 0.  0.  0.  0.  0.  0.5 0.5 0.  0.  0.  0.  0.  0.  0.5 0.  0.  0.5
 0.5 1.  0.  0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.  1.  0.5 0.  0.  0.  1.5]
predictions: [1. 1. 1. 1. 1. 1. 0. 0. 1. 0. 1. 1. 1. 0. 0. 1. 1. 1. 0. 0. 0. 1. 1. 1.
 1. 1. 0. 1. 0. 1. 0. 1. 0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 0. 0.
 1. 0. 0. 1. 0. 1. 1. 0. 1. 1. 0. 1. 1. 1. 0. 1. 0. 1. 1. 1. 0. 1. 0. 1.
 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 0. 1. 0. 0. 0. 1. 1. 1. 1. 0. 0. 1. 1. 1.
 0. 1. 

In [6]:
sample_result.performance_metrics

{'quantile_score': array([0. , 0. , 0. , 0.5, 1. , 0.5, 0. , 0.5, 0.5, 1. , 0. , 0.5, 0.5,
        0. , 1. , 0.5, 0.5, 0.5, 0. , 0. , 1. , 0. , 0. , 0. , 1. , 0.5,
        1. , 0.5, 0. , 0.5, 0.5, 0.5, 0.5, 1. , 0.5, 1.5, 0. , 0.5, 0. ,
        0.5, 1.5, 0.5, 4. , 0.5, 0. , 0.5, 0. , 1. , 0.5, 0. , 0.5, 0.5,
        1.5, 0. , 0.5, 0.5, 0.5, 0.5, 2. , 0.5, 0.5, 0.5, 0.5, 0.5, 1. ,
        0. , 0.5, 0.5, 1. , 0.5, 1.5, 0.5, 0. , 0.5, 0.5, 0.5, 0. , 0.5,
        2. , 0. , 0.5, 0.5, 0.5, 0.5, 0. , 0. , 1. , 0. , 0. , 0.5, 0.5,
        0. , 0.5, 0. , 0. , 0.5, 1. , 0.5, 0.5, 0. , 1. , 0.5, 0.5, 0.5,
        0.5, 1. , 1. , 0. , 0.5, 0. , 0. , 0. , 0. , 0. , 0.5, 0.5, 0. ,
        0. , 0. , 0. , 0. , 0. , 0.5, 0. , 0. , 0.5, 0.5, 1. , 0. , 0.5,
        0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0. , 1. , 0.5, 0. , 0. , 0. , 1.5]),
 'predictions': array([1., 1., 1., 1., 1., 1., 0., 0., 1., 0., 1., 1., 1., 0., 0., 1., 1.,
        1., 0., 0., 0., 1., 1., 1., 1., 1., 0., 1., 0., 1., 0., 1., 0., 1.,
        1.

In [7]:
sample_result.performance_metrics.get('quantile_score').shape

(143,)

In [8]:
sample_result.performance_metrics.get('quantile_score').mean()

np.float64(0.46853146853146854)

In [9]:
sample_result.performance_metrics

{'quantile_score': array([0. , 0. , 0. , 0.5, 1. , 0.5, 0. , 0.5, 0.5, 1. , 0. , 0.5, 0.5,
        0. , 1. , 0.5, 0.5, 0.5, 0. , 0. , 1. , 0. , 0. , 0. , 1. , 0.5,
        1. , 0.5, 0. , 0.5, 0.5, 0.5, 0.5, 1. , 0.5, 1.5, 0. , 0.5, 0. ,
        0.5, 1.5, 0.5, 4. , 0.5, 0. , 0.5, 0. , 1. , 0.5, 0. , 0.5, 0.5,
        1.5, 0. , 0.5, 0.5, 0.5, 0.5, 2. , 0.5, 0.5, 0.5, 0.5, 0.5, 1. ,
        0. , 0.5, 0.5, 1. , 0.5, 1.5, 0.5, 0. , 0.5, 0.5, 0.5, 0. , 0.5,
        2. , 0. , 0.5, 0.5, 0.5, 0.5, 0. , 0. , 1. , 0. , 0. , 0.5, 0.5,
        0. , 0.5, 0. , 0. , 0.5, 1. , 0.5, 0.5, 0. , 1. , 0.5, 0.5, 0.5,
        0.5, 1. , 1. , 0. , 0.5, 0. , 0. , 0. , 0. , 0. , 0.5, 0.5, 0. ,
        0. , 0. , 0. , 0. , 0. , 0.5, 0. , 0. , 0.5, 0.5, 1. , 0. , 0.5,
        0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0. , 1. , 0.5, 0. , 0. , 0. , 1.5]),
 'predictions': array([1., 1., 1., 1., 1., 1., 0., 0., 1., 0., 1., 1., 1., 0., 0., 1., 1.,
        1., 0., 0., 0., 1., 1., 1., 1., 1., 0., 1., 0., 1., 0., 1., 0., 1.,
        1.

In [37]:
release_manager = ReleaseManager()
output_dir = Path("./xgb_releases_2")
release_path = release_manager.create_complete_release(
    experiment_results=results_lightning_std,  # Your ExperimentResults from pipeline
    base_output_dir=output_dir
  )

# Check Data in piepeline

In [14]:
sku = (715, 7)
loader = DataLoader(data_config)
loader.load_data(lazy=False)
dataset = loader.prepare_modeling_dataset([sku], ModelingStrategy.INDIVIDUAL)

In [15]:
loader = DataLoader(data_config)
# Don't call load_data() at all - let prepare_modeling_dataset do it
dataset = loader.prepare_modeling_dataset([sku], ModelingStrategy.INDIVIDUAL)

In [72]:
# Access data
X_train_pipeline = dataset.X_train
y_train_pipeline = dataset.y_train
X_test_pipeline = dataset.X_test
y_test_pipeline = dataset.y_test

In [22]:
X_train_pipeline.shape

(741, 132)

# check data in prototype

In [23]:
import polars as pl
import pickle

In [34]:

# Data loading
mapping_path = "/Users/ivn/Documents/PhD/Transformer Research/Code/Benchmarking/data/feature_mapping_train.pkl"
features_path = "/Users/ivn/Documents/PhD/Transformer Research/Code/Benchmarking/data/db_snapshot_offsite/train_data/processed/train_data_features.feather"
target_path = "/Users/ivn/Documents/PhD/Transformer Research/Code/Benchmarking/data/db_snapshot_offsite/train_data/train_data/train_data_target.feather"

with open(mapping_path, 'rb') as f:
    mapping = pickle.load(f)

train_features = pl.read_ipc(features_path)
train_target = pl.read_ipc(target_path)

In [35]:
X = (train_features
     .lazy()
     .filter(pl.col('productID') == 715)
     .filter(pl.col('storeID') == 7)
     .drop_nulls()
     .sort("date","skuID")
     .collect())

X_bdIDs = (X
           .lazy()
           .select('bdID')
           .unique()
           .collect()
           .to_numpy()
           .flatten())

y = (train_target
     .lazy()
     .filter(pl.col("bdID").is_in(X_bdIDs))
     .join(
         X.lazy().select("bdID","skuID"), 
         on="bdID", 
         how="left")
     .sort("date","skuID")
     .collect())

In [36]:
split_date = pl.date(2016, 1, 1)
X_train = X.filter(pl.col("date") < split_date)
X_test = X.filter(pl.col("date") >= split_date)
y_train = y.filter(pl.col("date") < split_date)
y_test = y.filter(pl.col("date") >= split_date)

In [37]:
meta_cols = ['frequency',
 'idx',
 'bdID',
 'base_date',
 'date',
 'dateID',
 'skuID',
 'productID',
 'storeID',
 'companyID',
 'is_daily',
 'missing_value',
 'not_for_sale',
 'name',
 'name-2']

In [38]:
# Explicitly set dtype to float64 for all arrays
X_train = (X_train
            .sort("date","skuID")
            .select(pl.selectors.exclude(meta_cols)))

y_train = (y_train
            .sort("date","skuID")
            .select("target")
            .to_numpy())  # Also flatten to ensure 1D

X_test = (X_test
        .sort("date","skuID")
        .select(pl.selectors.exclude(meta_cols)))

y_test = (y_test
        .sort("date","skuID")
        .select("target"))

In [ ]:
#wrap them into print statements to see the output
print("Train set shape test:   ", X_train.shape == X_train_pipeline.shape )
print("Train label shape test: ", y_train.shape == y_train_pipeline.shape )
print(30*"-")
print("Test set shape test:  ", X_test.shape == X_test_pipeline.shape )
print("Test label shape test:", y_test.shape == y_test_pipeline.shape ) 

Train set shape test:    True
Train label shape test:  True
------------------------------
Test set shape test:   True
Test label shape test: True


In [ ]:
#test if prototype and pipeline data are the same
print("Training set check:    ", (X_train == X_train_pipeline).to_numpy().all())
print("Training label check:  ",(y_train == y_train_pipeline).all())
print(30*"-")
print("Test set check:   ", (X_test == X_test_pipeline).to_numpy().all())
print("Test label check: ",(y_test == y_test).to_numpy().all())  

Training set check:     True
Training label check:   True
------------------------------
Test set check:    True
Test label check:  True
